# Welcome to PYNQ

## Getting Started

To get started using PYNQ, try running the example notebooks in the folders described below. 

* **getting_started**: includes an introduction to using Jupyter notebook with PYNQ, the Python environment, and how to use some basic features of the current platform. 

* **common**: contains example notebooks on how to download an overlay, how to set the Zynq clocks, how to execute Linux shell commands, and how to use USB devices.

If other overlays or packages are installed, other folders with example notebooks may also be available in this directory.  


## Documentation

Please see the latest <a href="http://pynq.readthedocs.io">PYNQ Documentation on readthedocs</a>.  


## Support

For questions or support, go to the forum on the <a href="http://www.pynq.io">PYNQ project webpage </a>.


## Project webpage

You can find details on the <a href="http://www.pynq.io">PYNQ project webpage </a>.


## GitHub

The PYNQ Repository is hosted on github: <a href="https://github.com/Xilinx/PYNQ">PYNQ GitHub Repository</a>.

In [2]:
from pynq import Overlay
import pynq.lib.dma
from pynq import Xlnk
import numpy as np
import csv

ol = Overlay("../fpga2/design_1.bit")

ol?

In [5]:
import numpy as np
from pynq import allocate
from pynq import Overlay

dma = ol.axi_dma_0

input_row = 64
input_col = 7
hidden_row = 8
hidden_col = 2
sigmoid = 256
output_row = 3
output_col = 1
input_size = input_row * input_col + hidden_row * hidden_col + sigmoid + output_row * output_col
output_size = output_col * input_row

input_buffer = allocate(shape=(input_size,), dtype=np.uint32)
output_buffer = allocate(shape=(output_size,), dtype=np.uint32)
print("Input size:", input_size, "Output size:", output_size)

Input size: 723 Output size: 64


In [6]:
import csv

i = 0

#read input layer
with open('../csv/X.csv') as f:
    f_csv = csv.reader(f)
    for row in f_csv:
        for item in row:
            input_buffer[i] = item
            i = i + 1
            pass
            
#read hidden layer
with open('../csv/w_hid.csv') as f:
    f_csv = csv.reader(f)
    for row in zip(*f_csv):
        for item in row:
            input_buffer[i] = item
            i = i + 1

#read sigmoid layer
with open('../csv/sigmoid.csv') as f:
    f_csv = csv.reader(f)
    for row in f_csv:
        for item in row:
            input_buffer[i] = item
            i = i + 1

#read output layer
with open('../csv/w_out.csv') as f:
    f_csv = csv.reader(f)
    for row in zip(*f_csv):
        for item in row:
            input_buffer[i] = item
            i = i + 1

In [26]:
import time
start_time = time.time()
dma.sendchannel.transfer(input_buffer)
dma.recvchannel.transfer(output_buffer)
dma.sendchannel.wait()
dma.recvchannel.wait()
stop_time = time.time()
acc_time = stop_time - start_time
print(output_buffer)

[100 153 152 135 122 127 152 160 146 128 120 118 166 153 153 128 167 123
 148 119 127 130 153 112 146 141 157 145 120 111 119 122 122 114 142 146
 121 157 115 160 115 187 117 116 129 161 177 139 153 121 111 143 142 118
 152 114 110 164 119 122 157 119 135 125]


In [28]:
import numpy as np
inputs = np.loadtxt('../csv/X.csv', delimiter=',', dtype ='int')
hidden = np.loadtxt('../csv/w_hid.csv', delimiter=',', dtype ='int')
hidden = np.transpose(hidden)
sigmoid = np.loadtxt('../csv/sigmoid.csv', delimiter=',', dtype ='int')
outputs = np.loadtxt('../csv/w_out.csv', delimiter=',', dtype ='int')
outputs = np.transpose(outputs)

In [29]:
results = list()
start_time = time.time()
for a in range(input_row):
    sums = [0 for x in range(hidden_col)]
    for c in range(hidden_col):
        for b in range(input_col):
            sums[c] = sums[c] + inputs[a][b] * hidden[c][b + 1]
        sums[c] = sums[c] >> 8
        sums[c] += hidden[c][0]    
        sums[c] = sigmoid[sums[c]]
    mul = [(sums[x] * outputs[x + 1]) >> 8 for x in range(hidden_col)]
    results.append(sum(mul) + outputs[0])
stop_time = time.time()
sw_time = stop_time - start_time

#validate results
print("FPGA time:", acc_time, "Software time:", sw_time)
print("Acceleration factor:", sw_time / acc_time)
miss = 0
for x in range(input_row):
    if results[x] != output_buffer[x]:
        miss = miss + 1
        print("Wrong! FPGA output:", output_buffer[x], "Real output:", results[x])
print("Accuracy: ", miss / input_row)

FPGA time: 0.002503633499145508 Software time: 0.011945962905883789
Acceleration factor: 4.771450338063041
Wrong! FPGA output: 135 Real output: 134
Wrong! FPGA output: 127 Real output: 126
Wrong! FPGA output: 152 Real output: 151
Wrong! FPGA output: 160 Real output: 159
Wrong! FPGA output: 166 Real output: 165
Wrong! FPGA output: 153 Real output: 152
Wrong! FPGA output: 128 Real output: 127
Wrong! FPGA output: 119 Real output: 118
Wrong! FPGA output: 130 Real output: 129
Wrong! FPGA output: 146 Real output: 145
Wrong! FPGA output: 145 Real output: 144
Wrong! FPGA output: 120 Real output: 119
Wrong! FPGA output: 111 Real output: 110
Wrong! FPGA output: 119 Real output: 118
Wrong! FPGA output: 142 Real output: 141
Wrong! FPGA output: 121 Real output: 120
Wrong! FPGA output: 129 Real output: 128
Wrong! FPGA output: 161 Real output: 160
Wrong! FPGA output: 121 Real output: 120
Wrong! FPGA output: 111 Real output: 110
Wrong! FPGA output: 143 Real output: 142
Wrong! FPGA output: 142 Real out